In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
from datetime import datetime, timedelta, date
from dateutil import relativedelta as rd

In [2]:
# Upload DB
orders = pd.read_csv("orders_20190822.csv",sep=";",decimal=",",parse_dates=["o_date"])

In [3]:
list(orders.columns)

['id_o', 'user_id', 'price', 'o_date']

In [4]:
# Rename columns
orders = orders.rename(columns={'id_o': 'id', 'user_id': 'user', 'o_date': 'date'})
list(orders.columns)

['id', 'user', 'price', 'date']

In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
id       int64
user     int64
price    float64
date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 61.1 MB


In [6]:
orders["date"].max()

Timestamp('2017-12-31 00:00:00')

In [7]:
orders["price"].max()

819096.6

In [8]:
orders["date"].min()

Timestamp('2016-01-01 00:00:00')

In [9]:
orders["price"].min()

-184.8

In [10]:
# We translate into date format and delete fake orders
orders['date'] = pd.to_datetime(orders.date)
orders = orders[~(orders['price']<=99) & ~(orders['price']>=300000)]

In [11]:
price_date = orders[["price","date"]]
price_date.head()

,price,date
0,539.0,2016-01-01
1,153.3,2016-01-01
3,752.5,2016-01-01
4,4410.0,2016-01-01
5,595.0,2016-01-01


In [12]:
# Settings
period_pred = (datetime(2017, 12, 1), datetime(2017, 12, 31)) #Period
days_lost = timedelta(180) # The number of days after which we think that the user is lost
CC_pred = 0 # Planned turnover
order_price_mean = orders['price'].mean()

In [13]:
FACT = orders[orders['date'].between(*period_pred)]['price'].sum()
FACT

322926463.3

In [14]:
# Active users who are not considered lost
active_users = orders.groupby('user').filter(lambda o: o['id'].count() >= 3 and o['date'].max() > period_pred[0] - days_lost)

In [15]:
# Users who were active but stopped buying
lost_users = orders.groupby('user').filter(lambda o: o['id'].count() >= 3 and o['date'].max() <= period_pred[0] - days_lost)

In [16]:
# Users who made only 1 order
one_users = orders.groupby('user').filter(lambda o: o['id'].count() == 1)

In [17]:
# Users who made only 2 orders
two_users = orders.groupby('user').filter(lambda o: o['id'].count() == 2)

In [18]:
# Calculation check
len(one_users) + len(two_users) + len(active_users) + len(lost_users) == len(orders)

True

In [20]:
# Forecast volume
active_users_price = active_users.groupby('date')['price'].sum().mean()*(period_pred[1] - period_pred[0]).days
active_users_price

72764556.17031464

In [21]:
# Seasonality factor from past lesson
season_coefs = [0.98050609, 0.87231913, 1.03665563, 1.04224875, 0.90690877,
       0.82487778, 0.77827074, 0.8672703 , 0.88338003, 1.14733963,
       1.31235957, 1.34786358]
season_K  = season_coefs[period_pred[0].month - 1]
season_K

1.34786358

In [22]:
# Forecast for new users
one_users_mean_order = one_users['price'].mean() # average order volume for new users
period_fact = [d - rd.relativedelta(months=12) for d in period_pred]

users_ly = one_users[one_users['date'].between(*period_fact)]['id'].count()
users_ly_1 = one_users[one_users['date'].between(*[d - rd.relativedelta(months=1) for d in period_fact])]['id'].count()
users_lm = one_users[one_users['date'].between(*[d - rd.relativedelta(months=1) for d in period_pred])]['id'].count()
k = users_ly / users_ly_1 # Index
k

1.156953103822174

In [23]:
one_users_price = users_lm * k * one_users_mean_order

one_users_price

148540302.39466053

In [24]:
# Consider users with two orders. Determine the average time between two orders.
# We will find users who actually belong to the group of people with one order
# After we make a forecast for these two groups
left = pd.DataFrame(two_users.groupby('user')['date'].min())
right= pd.DataFrame(two_users.groupby('user')['date'].max())
min_max_dates = left.merge(right, left_on='user', right_on='user')
min_max_dates = min_max_dates.rename(columns={'date_x': 'min', 'date_y': 'max'})

In [25]:
# Real users with two orders
two_users_ids = min_max_dates[~(min_max_dates['min'] == min_max_dates['max'])] 

In [27]:
# Other users who have made two orders in a row and relate to users with one order
# Let's make a prediction on them, as before
one_users_more = len(min_max_dates) - len(two_users_ids)
one_users_price_more = one_users_more * k * one_users_mean_order
one_users_price_more

81479157.40935278

In [28]:
return_days = (two_users_ids['max'] - two_users_ids['min']).mean().days
return_days

145

In [29]:
# Real orders for users with two orders
two_users = two_users[two_users['user'].isin(two_users_ids.index)]

In [30]:
# We calculate the probability that a user with one order will buy again
p_from_one = len(two_users)/(len(one_users) + one_users_more + len(two_users))
p_from_one

0.1556452477952605

In [32]:
# Consider how many users with one order will buy earlier in the forecast period
period_one_fact = [d - timedelta(return_days) for d in period_pred]
two_users_price = one_users[one_users['date'].between(*period_one_fact)]['id'].count() * \
                        p_from_one  * (1 + one_users_more / len(one_users)) * one_users_mean_order

two_users_price

12711824.6966826

In [33]:
# Calculate how many users from previously lost bought again, and calculate the probability of return
period_fact_lm = [d - rd.relativedelta(months=1) for d in period_pred]
lost_users_ly = orders[orders['date'] < period_fact_lm[1]].groupby('user').\
    filter(lambda o: o['id'].count() >= 2 and o['date'].max() <= period_fact_lm[0] - days_lost)

In [34]:
# The number of former active users who returned
lost_users_returned = orders.loc[orders['date'].between(*period_fact_lm) & \
                                 orders['user'].isin(lost_users_ly['user'].unique())]

In [35]:
p_lost_return = len(lost_users_returned)/len(lost_users_ly)
p_lost_return

0.0003607965074898075

In [36]:
lost_users_price = len(lost_users) * p_lost_return * k * order_price_mean
lost_users_price

209376.85943012533

In [37]:
TO = one_users_price + one_users_price_more + two_users_price + lost_users_price

In [38]:
print('Fact: {:.0f}, Forecast: {:.0f}. Error {:0.3%}'.format(FACT, TO * season_K, TO * season_K/FACT - 1))

Fact: 322926463, Forecast: 327450870. Error 1.401%
